In [1]:
%%time

import glob
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from sklearn.decomposition import PCA

def find_nth(strValue, strFind, intPos):
    start = strValue.find(strFind)
    while start >= 0 and intPos > 1:
        start = strValue.find(strFind, start+len(strFind))
        intPos -= 1
    return start

filelist = glob.glob('./Parkinson/*PubTator.txt')
content = ''
content_list = []
documents = []

for filename in filelist:

    with open(filename, 'r') as infile:
        content = infile.read()
    
    content_list = content.split('\n\n\n')

    for doc in content_list:

        if not doc:
            continue

        lines = doc.split('\n')
        pmid = lines[0].split("|")[0]

        title = lines[0].split('|')
        abstract = lines[1].split('|')

        if (len(title) == 3):
            title = title[-1]
        else:
            title = []
        if (len(abstract) == 3):
            abstract = abstract[-1]
        else:
            abstract = []

        documents.append(''.join(title) + ' '+ ''.join(abstract))
        
'''
is_noun = (lambda pos: pos[:2] == 'NN')
is_nounp = (lambda pos: pos[:2] == 'NNP')
'''
#wpt = [nltk.word_tokenize(doc) for doc in documents]
tokenizer = RegexpTokenizer("[\w]+")
wpt = [tokenizer.tokenize(doc) for doc in documents]

#tokenized_doc = [[word.lower() for (word, pos) in nltk.pos_tag(line)] for line in wpt]
tokenized_doc = [[word.lower() for word in line] for line in wpt]

# word2vec 계산
num_features = 300             # 문자 벡터 차원 수
min_word_count = 1             # 최소 문자 수
num_workers = 4                # 병렬 처리 스레드 수
context = 10                   # 문자열 창 크기
downsampling = 1e-3            # 문자 빈도수 Downsample

word2vec_model = Word2Vec(tokenized_doc, 
                          workers=num_workers, 
                          size=num_features, 
                          min_count=min_word_count, 
                          window=context, 
                          sample=downsampling) 
vocab = word2vec_model[word2vec_model.wv.vocab] 
word2vec_model.wv.save_word2vec_format('word2vec_pubtator.bin', binary = True) 
word2vec_model.init_sims(replace=True) # 메모리 초기화

C:\ProgramData\Anaconda3\envs\TensorFlow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\envs\TensorFlow\lib\site-packages\ipykernel_launcher.py:74: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Wall time: 3min 20s


In [3]:
similar_words = {search_term: [item[0] for item in word2vec_model.wv.most_similar([search_term], topn=100)]
                  for search_term in ['seizures']}
print(similar_words)

print(word2vec_model.most_similar(positive=['seizures'], negative = ['wdr45'], topn = 10))

{'seizures': ['epilepsy', 'myoclonic', 'myoclonus', 'convulsions', 'spasms', 'jerks', 'seizure', 'epileptic', 'spasticity', 'clonic', 'hypotonia', 'nystagmus', 'microcephaly', 'dysarthria', 'attacks', 'paroxysmal', 'episodes', 'hallucinations', 'delusions', 'hemiparesis', 'interictal', 'generalized', 'paralysis', 'tremors', 'hemiplegia', 'dystonic', 'convulsion', 'vertigo', 'discharges', 'headache', 'dyskinesia', 'choreoathetosis', 'apraxia', 'intractable', 'psychosis', 'areflexia', 'dysphagia', 'convulsive', 'dizziness', 'psychomotor', 'symptoms', 'cataplexy', 'headaches', 'tics', 'hyporeflexia', 'audiogenic', 'myokymia', 'dyskinesias', 'stereotypic', 'delirium', 'truncal', 'ptosis', 'ataxic', 'tachycardia', 'ophthalmoparesis', 'auras', 'hypotension', 'hyperkinetic', 'epilepsies', 'akinesia', 'hyperreflexia', 'gaze', 'gait', 'downbeat', 'weakness', 'disinhibition', 'movements', 'blindness', 'retinitis', 'syncope', 'disorientation', 'generalised', 'aggression', 'episodic', 'stridor', '

C:\ProgramData\Anaconda3\envs\TensorFlow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """
